# imports

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import user_functions as uf

# Variables

In [2]:
# cd to the directory where the data is stored
# %cd /Users/alireza_amani/Desktop/PhD_files/Lysimeter_project_info/DataVerse/processed/

# provide the paths to the data and the output directory
# note that this repo does not contain the data.
# you need to download the data from the dataverse and provide the path to the data directory here
DATA_DIR = Path(
    "../data/"
)

paths = {
    "data_percolation": DATA_DIR / Path("FieldHourlyDeepPercolation_3Enclosures_4Lysimeters_201806_202210.csv"),
    "data_meteo_hourly": DATA_DIR / Path("HourlyMeteo_201707_202212.parquet"),
    "data_meteo_daily": DATA_DIR / Path("DailyMeteo_201707_202212.csv"),
    "data_soil_moist": DATA_DIR / Path("FieldSoilMoisture_4Enclosures_4Lysimeters_201806_202210.parquet"),
    "data_soil_temp": DATA_DIR / Path("FieldSoilTemperature_4Enclosures_4Lysimeters_201806_202210.parquet"),

    # this is where we save the Lysimeter objects, to be used in the analysis notebook
    "save_dir": Path("../results/")

}


# assert
for path in paths.values():
    assert path.exists(), f"{path} does not exist"

# Reading

In [3]:
# read the data into pandas dataframes
## csv files
data_percolation = pd.read_csv(
    paths["data_percolation"], index_col=0, parse_dates=True,
)
data_meteo_daily = pd.read_csv(
    paths["data_meteo_daily"], index_col=0, parse_dates=True,
)

## parquet files
data_soil_moist = pd.read_parquet(paths["data_soil_moist"])
data_soil_temp = pd.read_parquet(paths["data_soil_temp"])
data_meteo_hourly = pd.read_parquet(paths["data_meteo_hourly"])

### making sure of the datetime
data_soil_moist["Date Time (UTC)"] = pd.to_datetime(
    data_soil_moist["Date Time (UTC)"], utc=True
)
data_soil_temp["Date Time (UTC)"] = pd.to_datetime(
    data_soil_temp["Date Time (UTC)"], utc=True
)
data_meteo_hourly["Date Time (UTC)"] = pd.to_datetime(
    data_meteo_hourly["Date Time (UTC)"], utc=True
)

data_soil_moist = data_soil_moist.set_index("Date Time (UTC)")
data_soil_temp = data_soil_temp.set_index("Date Time (UTC)")
data_meteo_hourly = data_meteo_hourly.set_index("Date Time (UTC)")

# Processing

## The L1 lysimeter

In [6]:
# preparing the time-series needed to create the LysimeterEvents object
l1_percolation_tseries = data_percolation.loc[
    data_percolation["Lysimeter Number"] == 1, "Reading"
].asfreq("h").copy() # necessary to explicitly set the frequency to hourly

rainfall_tseries = data_meteo_hourly.loc[
    data_meteo_hourly["Variable"] == "Rainfall", "Reading"
].asfreq("h").copy()

snowdepth_tseries = data_meteo_daily.loc[
    data_meteo_daily["Variable"] == "Snow depth", "Reading"
].copy()

## here we are using the soil moisture and temperature at the surface of the
## lysimeter to create the soil_sensors_df (75 mm depth)
surface_soil_tseries = pd.DataFrame({
    "Moisture": data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L1_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100),
    "Temperature": data_soil_temp.loc[
        data_soil_temp["Unique_label_"] == "L1_75mm_ext_Temperature_degc", "Reading"
    ],
})

# creating the LysimeterEvents object
L1_events = uf.LysimeterEvents(
    percolation_hourly_tseries=l1_percolation_tseries,
    rainfall_hourly_tseries=rainfall_tseries,
    snowdepth_cm_daily_tseries=snowdepth_tseries,
    soil_sensors_df=surface_soil_tseries,
)

## run the event detection method
L1_events.id_events(verbose=False, event_count_prefix="L1_Event_")

## run the rainfall event detection method
## (rainfall event refers to the rainfall associated with the percolation event)
L1_events.id_rainfall_events(verbose=False)

## this method calculates the snow-related information for each event
L1_events.calculate_snow(pre_window_days=4)

## this method parses the relevant data from the soil sensors, including
## antecedent soil moisture and temperature
L1_events.get_surface_soil_into(
    soil_moist_label="Moisture",
    soil_temp_label="Temperature",
)


## in the following lines, we adjust some of the values for antecedent soil
## moisture and temperature for soem of the events. This is done based on
## manual inspection of the data.

### copying the dataframes to make sure we are not modifying the original
events_df = L1_events.events_df.copy()
rainfall_events_df = L1_events.rainfall_events_df.copy()
snow_events_df = L1_events.snow_events_df.copy()
surface_soil_df = L1_events.surface_soil_df.copy()

### manual entries to prior vwc
surface_soil_df.loc["L1_Event_3", "prior_vwc"] = 16.1
surface_soil_df.loc["L1_Event_3", "prior_vwc_dt"] = pd.Timestamp("2018-12-12 18:00:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_3", "prior_temp"] = 0.8


surface_soil_df.loc["L1_Event_4", "prior_vwc"] = 15.1
surface_soil_df.loc["L1_Event_4", "prior_vwc_dt"] = pd.Timestamp("2018-12-20 19:00:00-09:00", tz="UTC")
surface_soil_df.loc["L1_Event_4", "prior_temp"] = 0.4

surface_soil_df.loc["L1_Event_8", "prior_vwc"] = 9.5
surface_soil_df.loc["L1_Event_8", "prior_vwc_dt"] = pd.Timestamp("2020-01-03 10:00:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_8", "prior_temp"] = -0.1

surface_soil_df.loc["L1_Event_10", "prior_vwc"] = 21.0
surface_soil_df.loc["L1_Event_10", "prior_vwc_dt"] = pd.Timestamp("2020-08-04 14:00:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_10", "prior_temp"] = 20.4

surface_soil_df.loc["L1_Event_11", "prior_vwc"] = 16.3
surface_soil_df.loc["L1_Event_11", "prior_vwc_dt"] = pd.Timestamp("2020-08-29 14:30:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_11", "prior_temp"] = 16.3

surface_soil_df.loc["L1_Event_13", "prior_vwc"] = 10.0
surface_soil_df.loc["L1_Event_13", "prior_vwc_dt"] = pd.Timestamp("2021-03-20 13:00:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_13", "prior_temp"] = -0.2

surface_soil_df.loc["L1_Event_14", "prior_vwc"] = 13.9
surface_soil_df.loc["L1_Event_14", "prior_vwc_dt"] = pd.Timestamp("2021-04-22 07:30:00-00:00", tz="UTC")
surface_soil_df.loc["L1_Event_14", "prior_temp"] = 4.6

# replace the attribute in the LysimeterEvents object
surface_soil_df["prior_vwc"] = surface_soil_df["prior_vwc"].astype(float)
surface_soil_df["prior_temp"] = surface_soil_df["prior_temp"].astype(float)
L1_events.surface_soil_df = surface_soil_df.copy()

# pickle L1_events
L1_events.to_pickle(paths["save_dir"] / "L1_events.pkl")

# del all objects created in this cell
del (
    l1_percolation_tseries, rainfall_tseries, snowdepth_tseries,
    surface_soil_tseries, L1_events, events_df, rainfall_events_df,
    snow_events_df, surface_soil_df
)

## The L2 lysimeter

In [7]:
# preparing the time-series needed to create the LysimeterEvents object
l2_percolation_tseries = data_percolation.loc[
    data_percolation["Lysimeter Number"] == 2, "Reading"
].asfreq("h") # necessary to explicitly set the frequency to hourly
rainfall_tseries = data_meteo_hourly.loc[
    data_meteo_hourly["Variable"] == "Rainfall", "Reading"
].asfreq("h")
snowdepth_tseries = data_meteo_daily.loc[
    data_meteo_daily["Variable"] == "Snow depth", "Reading"
]

## here we are using the soil moisture and temperature at the surface of the
## lysimeter to create the soil_sensors_df (75 mm depth)
surface_soil_tseries = pd.DataFrame({
    "Moisture": data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L2_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100),
    "Temperature": data_soil_temp.loc[
        data_soil_temp["Unique_label_"] == "L2_75mm_ext_Temperature_degc", "Reading"
    ],
})

# creating the LysimeterEvents object
L2_events = uf.LysimeterEvents(
    percolation_hourly_tseries=l2_percolation_tseries,
    rainfall_hourly_tseries=rainfall_tseries,
    snowdepth_cm_daily_tseries=snowdepth_tseries,
    soil_sensors_df=surface_soil_tseries,
)

## run the event detection method
L2_events.id_events(verbose=False, event_count_prefix="L2_Event_")

## run the rainfall event detection method
## (rainfall event refers to the rainfall associated with the percolation event)
L2_events.id_rainfall_events(verbose=False)

## this method calculates the snow-related information for each event
L2_events.calculate_snow(pre_window_days=4)

## this method parses the relevant data from the soil sensors, including
## antecedent soil moisture and temperature
L2_events.get_surface_soil_into(
    soil_moist_label="Moisture",
    soil_temp_label="Temperature",
)


## in the following lines, we adjust some of the values for antecedent soil
## moisture and temperature for soem of the events. This is done based on
## manual inspection of the data.

### copying the dataframes to make sure we are not modifying the original
events_df = L2_events.events_df.copy()
rainfall_events_df = L2_events.rainfall_events_df.copy()
snow_events_df = L2_events.snow_events_df.copy()
surface_soil_df = L2_events.surface_soil_df.copy()

# manual entries to prior vwc
surface_soil_df.loc["L2_Event_5", "prior_vwc"] = 18.4
surface_soil_df.loc["L2_Event_5", "prior_vwc_dt"] = pd.Timestamp("2019-06-02 07:30:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_5", "prior_temp"] = 14.7


surface_soil_df.loc["L2_Event_6", "prior_vwc"] = 17.5
surface_soil_df.loc["L2_Event_6", "prior_vwc_dt"] = pd.Timestamp("2019-10-17 06:30:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_6", "prior_temp"] = 9.3


surface_soil_df.loc["L2_Event_8", "prior_vwc"] = 11.8
surface_soil_df.loc["L2_Event_8", "prior_vwc_dt"] = pd.Timestamp("2019-12-08 16:30:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_8", "prior_temp"] = -0.3

surface_soil_df.loc["L2_Event_16", "prior_vwc"] = 15.1
surface_soil_df.loc["L2_Event_16", "prior_vwc_dt"] = pd.Timestamp("2020-09-29 21:30:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_16", "prior_temp"] = 18.1

surface_soil_df.loc["L2_Event_17", "prior_vwc"] = 14.1
surface_soil_df.loc["L2_Event_17", "prior_vwc_dt"] = pd.Timestamp("2021-06-06 13:00:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_17", "prior_temp"] = 20.6

surface_soil_df.loc["L2_Event_18", "prior_vwc"] = 12.8
surface_soil_df.loc["L2_Event_18", "prior_vwc_dt"] = pd.Timestamp("2021-06-14 14:00:00-00:00", tz="UTC")
surface_soil_df.loc["L2_Event_18", "prior_temp"] = 19.7


# replace the attribute in the LysimeterEvents object
surface_soil_df["prior_vwc"] = surface_soil_df["prior_vwc"].astype(float)
surface_soil_df["prior_temp"] = surface_soil_df["prior_temp"].astype(float)
L2_events.surface_soil_df = surface_soil_df.copy()

# pickle L2_events
L2_events.to_pickle(paths["save_dir"] / "L2_events.pkl")

### del all objects created in this cell
del (
    l2_percolation_tseries, rainfall_tseries, snowdepth_tseries,
    surface_soil_tseries, L2_events, events_df, rainfall_events_df,
    snow_events_df, surface_soil_df
)

## The L3 lysimeter

In [8]:
# preparing the time-series needed to create the LysimeterEvents object
l3_percolation_tseries = data_percolation.loc[
    data_percolation["Lysimeter Number"] == 3, "Reading"
].asfreq("h") # necessary to explicitly set the frequency to hourly
rainfall_tseries = data_meteo_hourly.loc[
    data_meteo_hourly["Variable"] == "Rainfall", "Reading"
].asfreq("h")
snowdepth_tseries = data_meteo_daily.loc[
    data_meteo_daily["Variable"] == "Snow depth", "Reading"
]

## here we are using the soil moisture and temperature at the surface of the
## lysimeter to create the soil_sensors_df (75 mm depth)
surface_soil_tseries = pd.DataFrame({
    "Moisture": data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L3_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100),
    "Temperature": data_soil_temp.loc[
        data_soil_temp["Unique_label_"] == "L3_75mm_ext_Temperature_degc", "Reading"
    ],
})

# creating the LysimeterEvents object
L3_events = uf.LysimeterEvents(
    percolation_hourly_tseries=l3_percolation_tseries,
    rainfall_hourly_tseries=rainfall_tseries,
    snowdepth_cm_daily_tseries=snowdepth_tseries,
    soil_sensors_df=surface_soil_tseries,
)

## run the event detection method
L3_events.id_events(verbose=False, event_count_prefix="L3_Event_")

## run the rainfall event detection method
## (rainfall event refers to the rainfall associated with the percolation event)
L3_events.id_rainfall_events(verbose=False)

## this method calculates the snow-related information for each event
L3_events.calculate_snow(pre_window_days=4)

## this method parses the relevant data from the soil sensors, including
## antecedent soil moisture and temperature
L3_events.get_surface_soil_into(
    soil_moist_label="Moisture",
    soil_temp_label="Temperature",
)


## in the following lines, we adjust some of the values for antecedent soil
## moisture and temperature for soem of the events. This is done based on
## manual inspection of the data.

### copying the dataframes to make sure we are not modifying the original
events_df = L3_events.events_df.copy()
rainfall_events_df = L3_events.rainfall_events_df.copy()
snow_events_df = L3_events.snow_events_df.copy()
surface_soil_df = L3_events.surface_soil_df.copy()

# manual entries to prior vwc
surface_soil_df.loc["L3_Event_8", "prior_vwc"] = 13.5
surface_soil_df.loc["L3_Event_8", "prior_vwc_dt"] = pd.Timestamp("2020-09-29 21:30:00-00:00", tz="UTC")
surface_soil_df.loc["L3_Event_9", "prior_temp"] = 18.5

# replace the attribute in the LysimeterEvents object
L3_events.surface_soil_df = surface_soil_df.copy()

# pickle L3_events
surface_soil_df["prior_vwc"] = surface_soil_df["prior_vwc"].astype(float)
surface_soil_df["prior_temp"] = surface_soil_df["prior_temp"].astype(float)
L3_events.to_pickle(paths["save_dir"] / "L3_events.pkl")

### del all objects created in this cell
del (
    l3_percolation_tseries, rainfall_tseries, snowdepth_tseries,
    surface_soil_tseries, L3_events, events_df, rainfall_events_df,
    snow_events_df, surface_soil_df
)

## The L4 lysimeter

In [9]:
# preparing the time-series needed to create the LysimeterEvents object
l4_percolation_tseries = data_percolation.loc[
    data_percolation["Lysimeter Number"] == 4, "Reading"
].asfreq("h") # necessary to explicitly set the frequency to hourly
rainfall_tseries = data_meteo_hourly.loc[
    data_meteo_hourly["Variable"] == "Rainfall", "Reading"
].asfreq("h")
snowdepth_tseries = data_meteo_daily.loc[
    data_meteo_daily["Variable"] == "Snow depth", "Reading"
]

## here we are using the soil moisture and temperature at the surface of the
## lysimeter to create the soil_sensors_df (75 mm depth)
surface_soil_tseries = pd.DataFrame({
    "Moisture": data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L4_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100),
    "Temperature": data_soil_temp.loc[
        data_soil_temp["Unique_label_"] == "L4_75mm_ext_Temperature_degc", "Reading"
    ],
})

# creating the LysimeterEvents object
L4_events = uf.LysimeterEvents(
    percolation_hourly_tseries=l4_percolation_tseries,
    rainfall_hourly_tseries=rainfall_tseries,
    snowdepth_cm_daily_tseries=snowdepth_tseries,
    soil_sensors_df=surface_soil_tseries,
)

## run the event detection method
L4_events.id_events(verbose=False, event_count_prefix="L4_Event_")

## run the rainfall event detection method
## (rainfall event refers to the rainfall associated with the percolation event)
L4_events.id_rainfall_events(verbose=False)

## this method calculates the snow-related information for each event
L4_events.calculate_snow(pre_window_days=4)

## this method parses the relevant data from the soil sensors, including
## antecedent soil moisture and temperature
L4_events.get_surface_soil_into(
    soil_moist_label="Moisture",
    soil_temp_label="Temperature",
)


## in the following lines, we adjust some of the values for antecedent soil
## moisture and temperature for soem of the events. This is done based on
## manual inspection of the data.

### copying the dataframes to make sure we are not modifying the original
events_df = L4_events.events_df.copy()
rainfall_events_df = L4_events.rainfall_events_df.copy()
snow_events_df = L4_events.snow_events_df.copy()
surface_soil_df = L4_events.surface_soil_df.copy()

# manual entries to prior vwc
surface_soil_df.loc["L4_Event_9", "prior_vwc"] = 10.0
surface_soil_df.loc["L4_Event_9", "prior_vwc_dt"] = pd.Timestamp("2019-12-08 17:30:00-00:00", tz="UTC")
surface_soil_df.loc["L4_Event_9", "prior_temp"] = 0.2


surface_soil_df.loc["L4_Event_14", "prior_vwc"] = 16.8
surface_soil_df.loc["L4_Event_14", "prior_vwc_dt"] = pd.Timestamp("2020-10-10 15:00:00-00:00", tz="UTC")
surface_soil_df.loc["L4_Event_14", "prior_temp"] = 7.9


# replace the attribute in the LysimeterEvents object
surface_soil_df["prior_vwc"] = surface_soil_df["prior_vwc"].astype(float)
surface_soil_df["prior_temp"] = surface_soil_df["prior_temp"].astype(float)
L4_events.surface_soil_df = surface_soil_df.copy()


# pickle L4_events
L4_events.to_pickle(paths["save_dir"] / "L4_events.pkl")

### del all objects created in this cell
del (
    l4_percolation_tseries, rainfall_tseries, snowdepth_tseries,
    surface_soil_tseries, L4_events, events_df, rainfall_events_df,
    snow_events_df, surface_soil_df
)

# Manual verification of prior vwc

In [ ]:
from matplotlib import pyplot as plt

# font parameters
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'

## Example

In [ ]:
# load the LysimeterEvents objects for one of the lysimeters
L1_events = pd.read_pickle(paths["save_dir"] / "L1_events.pkl")

# lets choose an event for L1
event_index = "L1_Event_14"
event_start_dt, event_end_dt = L1_events.events_df.loc[
    event_index, ["event_start_dt", "event_end_dt"]
]
pre_window_dt = event_start_dt - pd.Timedelta(days=4)

rainfall_start_dt, rainfall_end_dt = L1_events.rainfall_events_df.loc[
    event_index, ["rainfall_start_dt", "rainfall_end_dt"]
]

ant_soil_moist, ant_soil_moist_dt = L1_events.surface_soil_df.loc[
    event_index, ["prior_vwc", "prior_vwc_dt"]
]


# plot the event
FIGSIZE = (14, 14)
BARWDTH = 0.1
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# plot as bar plot rainfall on first row
ax[0].bar(
    data_meteo_hourly.loc[
        data_meteo_hourly["Variable"] == "Rainfall", "Reading"
    ].loc[pre_window_dt:rainfall_end_dt].index,
    data_meteo_hourly.loc[
        data_meteo_hourly["Variable"] == "Rainfall", "Reading"
    ].loc[pre_window_dt:rainfall_end_dt],
    width=BARWDTH,
    color="blue",
    label="Rainfall",
)

# plot surface soil moisture on second row
ax[1].plot(
    data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L1_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100).loc[pre_window_dt:event_end_dt].index,
    data_soil_moist.loc[
        data_soil_moist["Unique_label_"] == "L1_75mm_ext_Moisture_percent", "Reading"
    ].multiply(100).loc[pre_window_dt:event_end_dt],
    color="black",
    label="Surface soil moisture",
)

# plot percolation on third row
ax[2].bar(
    data_percolation.loc[
        data_percolation["Lysimeter Number"] == 1, "Reading"
    ].loc[event_start_dt:event_end_dt].index,
    data_percolation.loc[
        data_percolation["Lysimeter Number"] == 1, "Reading"
    ].loc[event_start_dt:event_end_dt],
    width=BARWDTH,
    color="black",
    label="Percolation",
)

# plot the antecedent soil moisture
## if not NaT
if not pd.isna(ant_soil_moist_dt):
    ax[1].axvline(
        ant_soil_moist_dt,
        color="red",
        linestyle="--",
        label="Antecedent soil moisture",
)


# labels
ax[0].set_ylabel("Rainfall (mm)")
ax[1].set_ylabel("Soil moisture (%)")
ax[2].set_ylabel("Percolation (mm)")

# legend
ax[0].legend(loc="upper left", fontsize=13)
ax[1].legend(loc="upper right", fontsize=13)
ax[2].legend(loc="upper left", fontsize=13)


# xticklabel only for t0 and t1
ax[2].set_xticks([pre_window_dt, event_end_dt])

plt.show()